In [1]:
import sys
import warnings
warnings.filterwarnings('ignore')
import glob
!{sys.executable} -m pip install numpy pandas matplotlib scikit-learn pyarrow seaborn imblearn xgboost| grep -v 'already satisfied'

#

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.


In [2]:
df = pd.read_csv("data/train.csv")

In [3]:
df.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7905 entries, 0 to 7904
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             7905 non-null   int64  
 1   N_Days         7905 non-null   int64  
 2   Drug           7905 non-null   object 
 3   Age            7905 non-null   int64  
 4   Sex            7905 non-null   object 
 5   Ascites        7905 non-null   object 
 6   Hepatomegaly   7905 non-null   object 
 7   Spiders        7905 non-null   object 
 8   Edema          7905 non-null   object 
 9   Bilirubin      7905 non-null   float64
 10  Cholesterol    7905 non-null   float64
 11  Albumin        7905 non-null   float64
 12  Copper         7905 non-null   float64
 13  Alk_Phos       7905 non-null   float64
 14  SGOT           7905 non-null   float64
 15  Tryglicerides  7905 non-null   float64
 16  Platelets      7905 non-null   float64
 17  Prothrombin    7905 non-null   float64
 18  Stage   

In [5]:
# create train and validation sets that are divided equally

In [6]:
from sklearn.model_selection import train_test_split

train, validation = train_test_split(df, stratify=df["Status"], test_size=0.2)

In [7]:
numeric_values = ["N_Days","Age","Bilirubin","Cholesterol","Albumin","Copper","Alk_Phos","SGOT","Tryglicerides","Platelets","Prothrombin","Stage"]

In [8]:
categorical_values = ["Drug","Sex","Ascites","Hepatomegaly","Spiders","Edema"]
categorical_bin_values = ["Drug","Sex","Ascites","Hepatomegaly","Spiders"]
categorical_oh_values = ["Edema"]

In [9]:
features_to_scale = ["N_Days","Age","Bilirubin","Cholesterol","Albumin","Copper","Alk_Phos","SGOT","Tryglicerides","Platelets","Prothrombin","Stage"]

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder

# numeric pipeline
num_pipeline = Pipeline([('func_transformer', FunctionTransformer(np.log1p)), 
                         ('minmax_scaler', MinMaxScaler()), ])




In [11]:
from sklearn.base import BaseEstimator, TransformerMixin 
class BinaryEncoder (BaseEstimator, TransformerMixin):
    def __init__ (self, features):
        self.features = features
    def fit(self, X, y=None): 
        return self # do nothing
    def transform(self, X):
        for c in self.features:
            values = X[c].unique()
            X[c].replace(values,(0,1), inplace=True)
        return X
        
        

In [12]:
# full pipeline (numeric & categorical)
full_pipeline = ColumnTransformer([ ("num", num_pipeline, numeric_values), 
                                   ("bin_cat", BinaryEncoder(categorical_bin_values), categorical_bin_values), 
                                   ("cat", OneHotEncoder(sparse_output=False), categorical_oh_values), ]) 
# train_prepared = full_pipeline.fit_transform(train)

In [13]:
# transform with pipeline
train_prepared = full_pipeline.fit_transform(df)

In [14]:
pd.DataFrame(train_prepared).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.669095,0.738826,0.300025,0.358523,0.597161,0.743104,0.441963,0.670052,0.220476,0.837508,0.105411,0.756471,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.868732,0.635766,0.122222,0.411010,0.663473,0.534588,0.414574,0.568644,0.335416,0.797707,0.284055,0.756471,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.929185,0.327178,0.385274,0.338005,0.666886,0.686385,0.327745,0.525664,0.141331,0.527019,0.372385,1.000000,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
3,0.868895,0.598065,0.066874,0.280415,0.649746,0.517531,0.450223,0.344878,0.365419,0.663933,0.244610,0.756471,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.619075,0.504140,0.154455,0.392184,0.700607,0.534588,0.363340,0.543485,0.365419,0.710477,0.231236,1.000000,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0


In [15]:
from imblearn.over_sampling import SMOTE

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE to the training data
X_train_smote, y_train_smote = smote.fit_resample(train_prepared, df["Status"])


In [16]:
# Train Model
from xgboost import XGBClassifier
    # Label Encoding
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels_en = label_encoder.fit_transform(y_train_smote)
model_xgb = XGBClassifier(random_state=42).fit(X_train_smote, labels_en)


In [17]:
# Cross Validation
from sklearn.model_selection import KFold, cross_val_score
xgb_scores = cross_val_score(model_xgb, X_train_smote, labels_en, scoring="neg_log_loss", cv=5)
print(-xgb_scores)
print(-xgb_scores.mean())

[0.37211166 0.32963774 0.29532586 0.25750316 0.24250234]
0.29941615116618003


In [18]:
# Hyperparameter tuning
from sklearn.model_selection import GridSearchCV

# Declare a parameter grid for GridSearch
param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [3,5,7],
    'subsample': [0.6, 0.8]
}

# GridSearchCV on RF+SMOTE classifier model (with SMOTE)
grid_search = GridSearchCV(XGBClassifier(random_state=42), param_grid, cv=5, scoring='precision', return_train_score=True)
grid_search.fit(X_train_smote, labels_en)

# Save best model from grid search
best_xgb = grid_search.best_estimator_



In [46]:
# # prepare validation data
# val_prepared = full_pipeline.fit_transform(validation)
# #     prepare labels
# val_labels_en = label_encoder.fit_transform(validation["Status"])

In [19]:
xgb_scores_final = cross_val_score(best_xgb, X_train_smote, labels_en, scoring="neg_log_loss", cv=5)
print(-xgb_scores_final)
print(-xgb_scores_final.mean())

[0.47366503 0.41797677 0.41251533 0.42019664 0.41807026]
0.42848480505492564


In [40]:
test = pd.read_csv("data/test.csv")
test_prepared = full_pipeline.fit_transform(test)

In [41]:
# Predict on the validation set with the best model
y_pred_best_xgb = best_xgb.predict(test_prepared)

In [47]:
y_pred_de = label_encoder.inverse_transform(y_pred_best_xgb)
df_final = pd.DataFrame(y_pred_de, columns=['Status'])
df_final["id_new"] = test["id"].values # method to copy values from another dataframe found from a post by BENY at https://stackoverflow.com/questions/45747589/copying-a-column-from-one-dataframe-to-another-gives-nan-values [last accessed:14/3/24]
df_final.set_index('id_new', inplace=True)
df_final.index.name = "id"
df_final.head()
filename = "xgb_output.csv"
df_final.to_csv(filename)

In [53]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5271 entries, 7905 to 13175
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Status  5271 non-null   object
dtypes: object(1)
memory usage: 82.4+ KB


In [66]:
########## Taken from id5059_p2_240404_2.ipynb
# Choose a model and predict class probabilities using the processed test data
def test_predict_proba(model, X_test_processed):
    predicted_probabilities = model.predict_proba(X_test_processed)

    # Create the submission DataFrame
    submission_df = pd.DataFrame({
        'id': test["id"],
        'Status_C': predicted_probabilities[:, 0],
        'Status_CL': predicted_probabilities[:, 1],
        'Status_D': predicted_probabilities[:, 2]
    })
    
    return submission_df, predicted_probabilities

def save_test_predictions(submission_df):

    # Format file name with the path
    submission_file_name = "xgb_pred.csv"

    # Save submission DataFrame to the formatted file name
    submission_df.to_csv(submission_file_path, index=False)
    return
    # Print the path (for reference)
#     return print(f"Predicted class probabilities for: {model_name_and_version}",'\n',f"File saved to: {submission_file_path}")

In [69]:
final_output_df = test_predict_proba(best_xgb, test_prepared)
final_output_df
# save_test_predictions(final_output_df)

(         id  Status_C  Status_CL  Status_D
 0      7905  0.225014   0.032450  0.742536
 1      7906  0.233511   0.475618  0.290871
 2      7907  0.022767   0.009941  0.967292
 3      7908  0.808454   0.006773  0.184773
 4      7909  0.274718   0.560426  0.164855
 5      7910  0.878916   0.028788  0.092295
 6      7911  0.872980   0.038812  0.088208
 7      7912  0.140089   0.086322  0.773590
 8      7913  0.035689   0.000584  0.963727
 9      7914  0.811975   0.006823  0.181202
 10     7915  0.622382   0.036886  0.340733
 11     7916  0.377804   0.005837  0.616359
 12     7917  0.213731   0.523565  0.262704
 13     7918  0.616597   0.020107  0.363296
 14     7919  0.588863   0.172982  0.238155
 15     7920  0.732979   0.007248  0.259774
 16     7921  0.281092   0.195012  0.523896
 17     7922  0.012210   0.003650  0.984140
 18     7923  0.699786   0.003206  0.297008
 19     7924  0.107771   0.778283  0.113947
 20     7925  0.002502   0.000338  0.997159
 21     7926  0.850765   0.00116

In [48]:
from sklearn.metrics import log_loss

best_score = log_loss(validation["Status"], y_val_pred_best_xgb)
print(best_score)

0.5894552609693242


In [31]:
log_loss(validation["Status"], model_xgb.predict_proba(val_prepared))


0.6071666790845676

In [ ]:
# Label Encoding
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels_en = label_encoder.fit_transform(train["Status"])
# k-fold (e.g. 5) validation, with log loss, scores need to be un-negated back
# scores = cross_val_score(model, train_prepared, labels_en, scoring="neg_log_loss", cv=5)
